In [1]:
# working dir
import os
import sys

cwd = os.getcwd()
root_dir = os.path.dirname(os.path.dirname(cwd))
sys.path.append(root_dir)

print(root_dir)

c:\Users\david\Desktop\Main\03_Uni\WWI21DSA\02_Vorlesungen\06_Projektrealisierung\Projektrealisierung


In [2]:
# Imports and settings
import pandas as pd
import random

random.seed(42)

pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)
pd.set_option("display.max_colwidth", None)

In [3]:
# Load data
abcd = pd.read_csv("../../src/data/ABCD_tripfiles_preprocessed.csv")
mnop = pd.read_csv("../../src/data/MNOP_tripfiles_preprocessed.csv")
zyxw = pd.read_csv("../../src/data/ZYXW_tripfiles_preprocessed.csv")
datasets = {"abcd": abcd, "mnop": mnop, "zyxw": zyxw}

C:\Users\david\AppData\Local\Temp\ipykernel_17532\1085071673.py:2: DtypeWarning: Columns (0,13,14,15,16,17,18,19,20,21,22,23,25,26) have mixed types. Specify dtype option on import or set low_memory=False.
  abcd = pd.read_csv("../../src/data/ABCD_tripfiles_preprocessed.csv")


In [4]:
meta = ['id', 'creation_time', 'airline_code', 'flight_number', 'flight_date',
       'departure_airport', 'user_name', 'action_name']

message = ['header_line', 'entry_details']

header_cols = ['header_category', 'header_id']

extracted_cols = ['scheduleState',
       'departureAirport', 'departureTime', 'arrivalAirport', 'arrivalTime',
       'aircraftRegistration', 'aircraftSubtype', 'aircraftVersion',
       'serviceType', 'nonOperational', 'flifoCancel', 'sequenceNumber',
       'bestTimes', 'mvtActual']

weight_cols = ['START_WI_weight', 'START_WI_index',
       'DO_WI_weight', 'DO_WI_index', 'PAX_WI_weight', 'PAX_WI_index',
       'TOTAL_DEADLOAD_WI_weight', 'TOTAL_DEADLOAD_WI_index', 'TOTAL_LOAD_WI',
       'TOTAL_TRAFFIC_LOAD', 'FUEL_INDEX', 'AZFW', 'ATOW', 'ALAW', 'ATXW',
       'AFT_LIMIT_ZFW', 'FWD_LIMIT_ZFW', 'AFT_LIMIT_TOW', 'FWD_LIMIT_TOW',
       'AFT_LIMIT_LAW', 'FWD_LIMIT_LAW', 'LIZFW', 'LITOW', 'LILAW',
       'MAC_AT_ZFW', 'MAC_AT_TOW', 'MAC_AT_LAW', 'DEADLOAD_MAC', 'UNDERLOAD',
       'LIMITING_WEIGHT', 'ALLOWED_TOW', 'ALLOWED_ZFW', 'ALLOWED_LAW',
       'ALLOWED_TXW', 'STABTO', 'OPTIMAL_TRIM', 'IDEAL_ADDITIONAL_LOAD_AFT',
       'IDEAL_ADDITIONAL_LOAD_FWD', 'TAIL_TIPPING_WI_weight',
       'TAIL_TIPPING_WI_index', 'TAIL_TIPPING_INDEX_EXCEEDED',
       'FWD_MOVABLE_PAX', 'AFT_MOVABLE_PAX', 'INDEX_OUT_OF_BALANCE',
       'LOAD_TO_AFT', 'LOAD_TO_FWD', 'ESTIMATED_TRAFFIC_LOAD', 'ESTIMATED_ZFW',
       'DELTA_ZFW', 'ZFW_TOLERANCE_EXCEEDED']


In [70]:
def create_flight_id(df):
    df["flight_id"] = df["departureAirport"] + df["arrivalAirport"] + df["departureTime"] + df["arrivalTime"]

create_flight_id(abcd)

In [71]:
joined_action_count = pd.DataFrame(columns=['dataset', 'action_name', 'count', 'percent'])
for name, dataset in datasets.items():
    action_count_series = dataset.drop_duplicates(subset=['flight_id', 'action_name'])['action_name'].value_counts().reset_index()
    action_count_df = pd.DataFrame(action_count_series)
    action_count_df["dataset"] = name
    action_count_df["percent"] = action_count_df["count"] / dataset["flight_id"].nunique()
    # append
    joined_action_count = pd.concat([joined_action_count, action_count_df])

joined_action_count.query("percent > 0.1")

C:\Users\david\AppData\Local\Temp\ipykernel_14968\155786409.py:8: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  joined_action_count = pd.concat([joined_action_count, action_count_df])


,dataset,action_name,count,percent
0,abcd,ASMMsgProcessor,2379,1.000000
1,abcd,UpdateFlightAction,1997,0.839428
2,abcd,AssignLCCAction,1997,0.839428
3,abcd,StoreRegistrationAndConfigurationAc,1996,0.839008
4,abcd,UpdateCrewDataAction,1992,0.837327
5,abcd,CalculateWeightAndTrimAction,1992,0.837327
6,abcd,ResetLoadingListRecordsAction,1489,0.625893
7,abcd,UpdateLoadTableAction,1489,0.625893
8,abcd,TransferCargoAction,964,0.405212
9,abcd,ChangeFlightLegStateAction,944,0.396805


In [72]:
top_x = 6
top_x_sets = []
for dataset_name in datasets.keys():
    top_x_set = joined_action_count[joined_action_count["dataset"] == dataset_name].sort_values("count", ascending=False).head(top_x)["action_name"].to_list()   
    top_x_sets.append(set(top_x_set))
    print(dataset_name, top_x_set)

relevant_actions_preliminary = top_x_sets[0].intersection(top_x_sets[1]).intersection(top_x_sets[2])

print(
    "Actions that are in the top {} for all datasets: ".format(top_x),
    relevant_actions_preliminary
    )

print(
    "Lowest percentage of actions in the top {} for all datasets: ".format(top_x),
    min(joined_action_count.query("action_name in @relevant_actions_preliminary")["percent"])
    )


# Take Away:
relevant_actions = ['ASMMsgProcessor', 'UpdateCrewDataAction', 'UpdateFlightAction', 'StoreRegistrationAndConfigurationAc', 'CalculateWeightAndTrimAction', 'AssignLCCAction']

abcd ['ASMMsgProcessor', 'UpdateFlightAction', 'AssignLCCAction', 'StoreRegistrationAndConfigurationAc', 'UpdateCrewDataAction', 'CalculateWeightAndTrimAction']
mnop ['ASMMsgProcessor', 'UpdateFlightAction', 'AssignLCCAction', 'CalculateWeightAndTrimAction', 'StoreRegistrationAndConfigurationAc', 'UpdateCrewDataAction']
zyxw ['ASMMsgProcessor', 'UpdateFlightAction', 'AssignLCCAction', 'StoreRegistrationAndConfigurationAc', 'CalculateWeightAndTrimAction', 'UpdateCrewDataAction']
Actions that are in the top 6 for all datasets:  {'AssignLCCAction', 'CalculateWeightAndTrimAction', 'ASMMsgProcessor', 'UpdateFlightAction', 'UpdateCrewDataAction', 'StoreRegistrationAndConfigurationAc'}
Lowest percentage of actions in the top 6 for all datasets:  0.7367953401125481


In [73]:
# For relevant actions: Check which header_categories are relevant
for action in relevant_actions:
    # for dataset_name, dataset in datasets.items():
    for cat in ['received', 'saved', 'sent']:
        filtered_df = dataset[
            (dataset["header_category"] == cat) &
            (dataset["action_name"] == action)
        ]
        (f"header_category == {cat} & action_name == {action}")
        print(dataset_name, action, cat)
        display(filtered_df[message].head(1))


# Take Away:
relevant_categories = {
    "CalculateWeightAndTrimAction": "saved",
    "AssignLCCAction": "saved",
    "StoreRegistrationAndConfigurationAc": "saved",
    "ASMMsgProcessor": "received",
    "UpdateCrewDataAction": "received",
    "UpdateFlightAction": "received"
}

zyxw ASMMsgProcessor received


,header_line,entry_details
29372,"2024-04-30 16:56:56,764 INFO [232ceb032f20e037] [ZY5975 /30 VCP | service-acco | ASMMsgProcessor | | Thread-28358 (ActiveMQ-client-global-threads)] Received:","<?xml version=""1.0"" encoding=""UTF-8""?><ns2:OSSChangeMessage xmlns:ns2=""http://www.systemone.com/iocc/oss/notification/OssChangeMessage-1.3.0"">\r\n<changeDescriptor>\r\n <changeDetail>\r\n <changeLocation>/OSSChangeMessage/newFlight/leg[1]/aircraftVersion</changeLocation>\r\n <changeDescription>\r\n <actionCode>CON</actionCode>\r\n </changeDescription>\r\n </changeDetail>\r\n <changeDetail>\r\n <changeLocation>/OSSChangeMessage/newFlight/leg[1]/aircraftRegistration</changeLocation>\r\n <changeDescription>\r\n <actionCode>EQT</actionCode>\r\n </changeDescription>\r\n </changeDetail>\r\n <changeDetail>\r\n <changeLocation>/OSSChangeMessage/newFlight/leg[1]/aircraftSubtype</changeLocation>\r\n <changeDescription>\r\n <actionCode>EQT</actionCode>\r\n </changeDescription>\r\n </changeDetail>\r\n </changeDescriptor>\r\n<oldFlight>\r\n <identity>\r\n <carrier>ZY</carrier>\r\n <dayOfOrigin>2024-04-30Z</dayOfOrigin>\r\n <number>5975</number>\r\n <id>30442698</id>\r\n <version>3</version>\r\n </identity>\r\n <leg>\r\n <scheduleState>SKD</scheduleState>\r\n <departureAirport>VCP</departureAirport>\r\n <departureTime>2024-04-30T21:00:00.000Z</departureTime>\r\n <arrivalAirport>JTC</arrivalAirport>\r\n <arrivalTime>2024-04-30T22:00:00.000Z</arrivalTime>\r\n <aircraftRegistration>ZYXQT</aircraftRegistration>\r\n <aircraftSubtype>AT9</aircraftSubtype>\r\n <aircraftVersion>Y70</aircraftVersion>\r\n <serviceType>J</serviceType>\r\n <nonOperational>false</nonOperational>\r\n <flifoCancel>false</flifoCancel>\r\n <sequenceNumber>1</sequenceNumber>\r\n <id>53360291</id>\r\n </leg>\r\n </oldFlight>\r\n<newFlight>\r\n <identity>\r\n <carrier>ZY</carrier>\r\n <dayOfOrigin>2024-04-30Z</dayOfOrigin>\r\n <number>5975</number>\r\n <id>30442698</id>\r\n <version>4</version>\r\n </identity>\r\n <leg>\r\n <scheduleState>SKD</scheduleState>\r\n <departureAirport>VCP</departureAirport>\r\n <departureTime>2024-04-30T21:00:00.000Z</departureTime>\r\n <arrivalAirport>JTC</arrivalAirport>\r\n <arrivalTime>2024-04-30T22:00:00.000Z</arrivalTime>\r\n <aircraftRegistration>ZYXXR</aircraftRegistration>\r\n <aircraftSubtype>E95</aircraftSubtype>\r\n <aircraftVersion>Y118</aircraftVersion>\r\n <serviceType>J</serviceType>\r\n <nonOperational>false</nonOperational>\r\n <flifoCancel>false</flifoCancel>\r\n <sequenceNumber>1</sequenceNumber>\r\n <id>53360291</id>\r\n </leg>\r\n </newFlight>\r\n</ns2:OSSChangeMessage>"


zyxw ASMMsgProcessor saved


,header_line,entry_details
29374,"2024-04-30 16:56:56,705 INFO [3bf7244b9a5c658a] [ZY5584 /30 VCP | service-acco | ASMMsgProcessor | | Thread-28359 (ActiveMQ-client-global-threads)] Saved:",The message was processed successfully


zyxw ASMMsgProcessor sent


...

,header_line,entry_details
63208,"2024-04-30 09:52:27,219 INFO [93c249c2005f62fc] [ZY2515 /30 LIS | human | AssignLCCAction | | default task-125] Received:",NaN


zyxw AssignLCCAction saved


,header_line,entry_details
63209,"2024-04-30 09:52:27,220 INFO [93c249c2005f62fc] [ZY2515 /30 LIS | human | AssignLCCAction | | default task-125] Saved:",A/L Flt No Suff Date DEP ARR STD in UTC ETD in UTC AC Type Reg Assigned user Planned user Location \r\nZY 2515 NULL 2024-04-30 LIS VCP 2024-04-30 15:00:00 2024-04-30 15:00:00 NULL NULL NULL NULL AZV


zyxw AssignLCCAction sent


,header_line,entry_details


In [74]:
# Function to filter the dataset based on relevant actions and categories
def filter_dataset(dataset):
    return dataset[
        (dataset['action_name'].isin(relevant_actions)) &
        (dataset.apply(lambda row: row['header_category'] == relevant_categories.get(row['action_name']), axis=1))
    ]

# Initialize and filter datasets in one step
trimmed_datasets = {dataset_name: filter_dataset(dataset) for dataset_name, dataset in datasets.items()}
for dataset_name in datasets.keys():
    print(f"{dataset_name}, vorher: {len(datasets[dataset_name])}, nachher: {len(trimmed_datasets[dataset_name])}")

abcd, vorher: 709613, nachher: 152220
mnop, vorher: 1108961, nachher: 235449
zyxw, vorher: 429759, nachher: 136051


In [75]:
joined_dataset = pd.concat([dataset[["flight_id"] + meta + message] for dataset in trimmed_datasets.values()])

In [76]:
# Check how much the dataset would be reduced if one only kept the flights that have all relevant actions at least once
grouped = joined_dataset.groupby('flight_id')
distinct_values = set(relevant_actions)

# Function to check if a group contains all distinct values
def has_distinct_values(group):
    return distinct_values.issubset(set(group['action_name']))

# Apply the function to each group and count the number of groups that meet the condition
count_groups_with_values = grouped.filter(has_distinct_values)['flight_id'].nunique()

print(joined_dataset["flight_id"].nunique())
print(count_groups_with_values)


13744
10509


In [77]:
for action in relevant_actions:
    print(action)
    entry_details = joined_dataset.loc[joined_dataset["action_name"] == action, "entry_details"]
    random_sample = entry_details.iloc[random.randrange(len(entry_details))]
    print(random_sample, end="\n\n")

ASMMsgProcessor
<?xml version="1.0" encoding="UTF-8"?><ns2:OSSChangeMessage xmlns:ns2="http://www.systemone.com/iocc/oss/notification/OssChangeMessage-1.3.0">
<changeDescriptor>
        <changeDetail>
            <changeLocation>/OSSChangeMessage/newFlight/leg[1]/acarsActual[1]/actualState</changeLocation>
            <changeLocation>/OSSChangeMessage/newFlight/leg[1]/acarsActual[1]/offblockTime</changeLocation>
            <changeLocation>/OSSChangeMessage/newFlight/leg[1]/acarsActual[1]/landingTime</changeLocation>
            <changeDescription>
                <movementState>OFF_BLOCK</movementState>
            </changeDescription>
        </changeDetail>
    </changeDescriptor>
<oldFlight>
        <identity>
            <carrier>ZY</carrier>
            <dayOfOrigin>2024-05-02Z</dayOfOrigin>
            <number>3983</number>
            <id>30447015</id>
            <version>3</version>
        </identity>
        <leg>
            <scheduleState>SKD</scheduleState>
            <

-------------

Vllt doch flight number als id

In [ ]:
# temp
cols = ["departureAirport", "arrivalAirport", "departureTime", "arrivalTime"]
zyxw[cols].head(2)

abcd[abcd["flight_id"].isna()].__len__()

In [64]:
df = abcd[abcd["flight_number"] == random.choice(abcd["flight_number"].unique())].sort_values("creation_time")[meta + ["header_category"]]

In [65]:
print(
    df["creation_time"].max(),
    df["creation_time"].min()
)

2024-05-07 08:42:34 2024-04-30 10:29:01


ne, macht keinen sinn